### Import the Libraries and Dataset

In [1]:
from sklearn2pmml.pipeline import PMMLPipeline
from sklearn2pmml import sklearn2pmml
import pandas as pd
import numpy as np

df = pd.read_csv('original_data.csv')
df.head()

,Code,Type,Length ave. (nm),Diameter ave. (nm),Diameter std,BET (m2/g),Fe content,Co content,Ni content,Mg content,...,12.5 (ug/ml)_prolif,25 (ug/ml)_prolif,50 (ug/ml)_prolif,100 (ug/ml)_prolif,200 (ug/ml)_prolif,CEA: C.H.N.O (wt%),OH mmol/g,COOH mmol/g,Endotoxins (EU/mg),Genotoxicity
0,NRCWE- 040,PRISTINE,518.9,22.1,7.8,150,0.200,0.001,0.56,0.01,...,95,85,60,67,62,96,0.35,0.18,0.18,0
1,NRCWE- 041,OH,1005.0,26.9,10.1,152,0.130,0.001,0.31,0.02,...,79,62,50,54,59,97,1.69,0.84,NaN,0
2,NRCWE- 042,COOH,723.2,30.2,14.2,141,0.080,0.000,0.21,0.03,...,89,96,72,79,56,96,4.09,2.04,0.26,0
3,NRCWE- 043,PRISTINE,771.3,55.6,18.1,82,0.008,0.001,1.20,0.01,...,105,115,114,111,101,96,0.18,0.09,0.25,0
4,NRCWE- 044,OH,1330.0,32.7,13.6,74,0.004,0.002,1.04,0.02,...,99,96,95,91,77,97,0.23,0.11,0.27,1


One-Hot encode the data using pandas get_dummies

In [2]:
df = pd.get_dummies(df)

We will drop the columns that we do not need for our model

In [3]:
all_columns = list(df.drop('Genotoxicity',axis=1).columns)
keep = ['Purity (%)','Zave (12.5 ug/ml)','PdI (batch)','Type_OH']

for i in all_columns:
    if i not in keep:
        df.drop(i,axis=1,inplace=True)

In [4]:
df.head()

,Purity (%),PdI (batch),Zave (12.5 ug/ml),Genotoxicity,Type_OH
0,98.6,0.358,159,0,0
1,99.2,0.501,236,0,1
2,99.2,0.382,155,0,0
3,98.5,0.237,145,0,0
4,98.6,0.236,437,1,1


### Instantiate and fit a scaling object

In [5]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(df)
scaled_data = pd.DataFrame(index=range(len(df)), columns=df.columns, data=scaler.transform(df))

c:\users\user\desktop\test-jupyter\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [6]:
scaled_data.head()

,Purity (%),PdI (batch),Zave (12.5 ug/ml),Genotoxicity,Type_OH
0,0.934783,0.349333,0.020378,0.0,0.0
1,1.000000,0.730667,0.132460,0.0,1.0
2,1.000000,0.413333,0.014556,0.0,0.0
3,0.923913,0.026667,0.000000,0.0,0.0
4,0.934783,0.024000,0.425036,1.0,1.0


### Load the final model

In [7]:
import joblib
logmodel = joblib.load('Final_model.sav')

In [8]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression(C=1374.54, penalty='l1', random_state=42)

### Extract the endpoint column

In [9]:
labels = df.pop('Genotoxicity')

### Create the PMML

In [10]:
pipeline = PMMLPipeline([("scale",scaler),("classifier",logmodel)])

In [11]:
pipeline.fit(df,labels);

c:\users\user\desktop\test-jupyter\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
c:\users\user\desktop\test-jupyter\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [12]:
sklearn2pmml(pipeline, "PMML_for_MWCNTs.pmml", with_repr = True);